# Pet Prediction

## Introduction

It happens more often than not - especially to animal & pet lovers such as myself - that you come across the most beautiful dog while going for a walk in the park -- or whenever you're mindlessly scrolling Facebook for hours. This project is a digitalised manifestation of your gung ho dog breed connoisseur from around the corner. How? Through the means of **Machine Learning** (ML).
The vast majority of blood, sweat and tears spent on researching and implementing the trained model will be documented by the use of the **Jupyter Notebook** you're currently reading.
Nevertheless, to take into account the target audience and pragmatism of this implementation, I have chosen to pair the aforementioned notebook with a frontend developed in React in order to evade the convoluted process of having to manually import images into the model -- which is still an option if you're into self-loathing.

The dataset used in this project is the [Animal Breed - Cats and Dogs](https://www.kaggle.com/imsparsh/animal-breed-cats-and-dogs) dataset from Kaggle.

## Imports

Your usual myriad of ML libraries were imported in this project such as:
- Numpy (Arrays & matrices) 
- Pandas (Manage the arrays)
- OS (Communication between your filesystem and python)
- Matplotlib (Plotting graphs)
- Tensorflow & Keras (Machine Learning)

Other than these I have also imported some lesser known libraries, and modules especially, to adapt to the specific circumstances of this project. These being:
- Preprocessing.image (Loading in images and transforming them into tensors)
- Sklearn.metrics (Visualising & reporting the model's performance & efficiency)
- Tensorflowjs (Exporting to a Javascript compatible model)

In [ ]:
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
import tensorflowjs as tfjs
from tensorflow import keras

## Setup

In order to adhere to coding conventions and to avoid confusion further on in the project I have tactically split up the setup part of my code from the rest in order to declare some constants and to initialise the mapping as well as the reverse mapping which will we'll take under the loop a bit farther down the project. 

In [ ]:
TRAIN = 'breeds/TRAIN'
TEST = 'breeds/TEST'
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
EPOCH_CNT = 3

dataset=[] 
name=[]
count=0
pet_dict=[]

Name = os.listdir(TRAIN)
N=list(range(len(Name)))
mapping = dict(zip(Name, N))
reverse_mapping = dict(zip(N, Name))

print(mapping)

## Data Cleaning & Processing



As this project is almost entirely image based there quite simply isn't a lot of cleaning, imputation or processing to be done.

In [ ]:

for file in sorted(os.listdir(TRAIN)):
    path=os.path.join(TRAIN,file)
    for im in os.listdir(path):
        image=load_img(os.path.join(path,im), grayscale=False, color_mode='rgb', target_size=(IMAGE_HEIGHT,IMAGE_WIDTH))
        image=img_to_array(image)
        image=image/255.0
        dataset.append([image,count])     
    count=count+1
test=[]
testfile=[]

for file in os.listdir(TEST):
    path=os.path.join(TEST,file)
    image=load_img(path, grayscale=False, color_mode='rgb', target_size=(IMAGE_HEIGHT,IMAGE_WIDTH))
    image=img_to_array(image)
    image=image/255.0
    test+=[image]
    testfile+=[file]
data,labels0=zip(*dataset)
labels1=to_categorical(labels0)
labels=np.array(labels1)
data=np.array(data)
test=np.array(test)
data2=data.reshape(-1,IMAGE_HEIGHT,IMAGE_WIDTH,3)
test2=test.reshape(-1,IMAGE_HEIGHT,IMAGE_WIDTH,3)
trainx,testx,trainy,testy=train_test_split(data,labels,test_size=0.2,random_state=44)
print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

## Training Model

In order to actually train the algorithm we'll make use of [**Densely Connected Convolutional Networks**](https://arxiv.org/abs/1608.06993) (DCCN). To go about this in a comprehensive and easy to use manner, we'll be implementing said DCCN using a Keras module which we've imported in one of the former stages of this document. 

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=20, zoom_range=0.2,
                             width_shift_range=0.2, height_shift_range=0.2, shear_range=0.1, fill_mode="nearest")

pretrained_model = tf.keras.applications.DenseNet121(input_shape=(
    IMAGE_HEIGHT, IMAGE_WIDTH, 3), include_top=False, weights='imagenet', pooling='avg')

pretrained_model.trainable = False

inputs = pretrained_model.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
outputs = tf.keras.layers.Dense(37, activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

his = model.fit(datagen.flow(trainx, trainy, batch_size=32),
                validation_data=(testx, testy), epochs=EPOCH_CNT)

## Model Fitness

In order to fathom the efficiency and as to how our newly born model performs we will be using a couple of built-in modules and functions from the Sklearn package. 
Previously the trained model is stored in the "his" variable. After going through the various learning cycles its accuracy and loss data are then stored in sepperate new variables and are then plotted in the next section. 

In [ ]:
y_pred = model.predict(testx)
pred = np.argmax(y_pred, axis=1)
ground = np.argmax(testy, axis=1)
print(classification_report(ground, pred))

get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

## Accuracy

In [ ]:
epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training VS Validation Accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

## Loss

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training VS Validation Loss')
plt.legend(loc=0)
plt.figure()
plt.show()

## Proof testing

This section is quite possibly the most gratifying part of this entire project. Sometimes at least. 
Firstly, we declare the reverse mapping of breeds (see #Setup) to then map the different representations of breeds - using integers - to their respective indeces. A random, yet cherry-picked for the sake of **proof** testing, is then loaded in and passed to the model.

In [ ]:
image=load_img("breeds/TEST/beagle/101.jpg",target_size=(256,256))
plt.imshow(image)
plt.show()

model = keras.models.load_model('models/model1.h5')

image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)

def mapper(value):
    return reverse_mapping[value]

## Prediction

In [ ]:
# PREDICTION
# print(prediction_image)
prediction=model.predict(prediction_image)
value=np.argmax(prediction)
move_name=mapper(value)
print(f"Prediction is {format(move_name).capitalize()}.")

## Serialisation

In order to make use of the functionality of the model it will be serialised as well as converted into a model which is able to be interpreted by the aforementioned React app. The TensorflowJS converter module will split up the model into one model.json as well as various weight files which will add their respective weights to the breeds. After this is done and dusted it is ready to be pushed straight to github in order for the raw model.json file to receive GET-requests.

In [ ]:
#Uncomment to save model & to export it to a tsjs compatible format.
with open('frontend/src/data/mapping.json', 'w') as mapping:
    json.dump(reverse_mapping, mapping)

model.save('models/model1.h5')
tfjs.converters.save_keras_model(model, 'models/tfjs')

## Conclusion

With an 89% accuracy one might say that this model is pretty spot on. There are still some times where the Keras model predicts something completely different from the selected image yet the instances where this does happen are mainly caused by the fact that the image has bad lighting. This is a big issue since colour is one of the hyperparameters and if the colour is not visible the model has a big problem predicting.

## Sources